In [1]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from scipy.io import loadmat

# Importing pyod packages and the methods

In [3]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging



# Import Performance matrix packages

In [4]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

# Define Data file and read X and Y

In [5]:
mat_file_list=['arrhythmia.mat',
               'cardio.mat',
               'glass.mat',
               'ionosphere.mat',
               'letter.mat',
               'lympho.mat',
               'mnist.mat',
               'musk.mat',
               'optdigits.mat',
               'pendigits.mat',
               'pima.mat',
               'satellite.mat',
               'satimage-2.mat',
               'shuttle.mat',
               'vertebral.mat',
               'vowels.mat',
               'wbc.mat']

# define nine outlier detection tools to be compared

In [6]:
df_columns=['Data','#Samples','#Dimensions','Outlier Perc',
            'ABOD','CBLOF','FB','HBOS','iForest','KNN','LOF','MCD',
            'OCSVM','PCA']
#roc (region of characteristic)performance evalutaion 
roc_df = pd.DataFrame(columns=df_columns)
#precision score performance evaluation
prn_df = pd.DataFrame(columns=df_columns)
#time dataframe performance evaluation
time_df =  pd.DataFrame(columns=df_columns)

# Exploring all mat files

In [30]:
from time import time
random_state=np.random.RandomState(42) 
for mat_file in mat_file_list:
    print("\n.....Processing",mat_file,"....")
    mat=loadmat(os.path.join('data',mat_file))
    
    X=mat['X']
    y=mat['y'].ravel()
    
    outliers_fraction=np.count_nonzero(y)/len(y)
    outliers_percentage=round(outliers_fraction*100,ndigits=4)
    #construct containers for saving result
    roc_list=[mat_file[:-4],x.shape[0],x.shape[1],outliers_percentage]
    prn_list=[mat_file[:-4],x.shape[0],x.shape[1],outliers_percentage]
    time_list=[mat_file[:-4],x.shape[0],x.shape[1],outliers_percentage]

     #60% data for training and 40% data for testing
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.4,random_state=random_state)

    #standardizing data for processing
    X_train_norm,X_test_norm=standardizer(X_train,X_test)

    classifiers={'Angle-based Outlier Detector(ABOD)':ABOD(contamination=outliers_fraction),
                 'Cluster-based Local Outlier Factor':CBLOF(contamination=outliers_fraction,check_estimator=False,random_state=random_state),
                 'Feature Bagging':FeatureBagging(contamination=outliers_fraction,random_state=random_state),
                 'Histogram-base Outlier Detection(HBOS)':HBOS(contamination=outliers_fraction),
                 'Isolation Forest':IForest(contamination=outliers_fraction,random_state=random_state),
                 'K Nearest Neighbors(KNN)':KNN(contamination=outliers_fraction),
                 'Local Outlier Factor(LOF)':LOF(contamination=outliers_fraction),
                 'Minimum Covariance Determinant(MCD)':MCD(contamination=outliers_fraction,random_state=random_state),
                 'One-class SVM(OCSVM)':OCSVM(contamination=outliers_fraction),
                 'Principal Component Analysis(PCA)':PCA(contamination=outliers_fraction,random_state=random_state)
                }
    for clf_name,clf in classifiers.items():
        t0=time()
        clf.fit(X_train_norm)
        test_scores=clf.decision_function(X_test_norm)
        t1=time()
        duration=round(t1-t0,ndigits=4)
        time_list.append(duration)
    
        roc=round(roc_auc_score(y_test,test_scores),ndigits=4)
        prn=round(precision_n_scores(y_test,test_scores),ndigits=4)
    
        print('{clf_name} ROC:{roc},precision @ rank n:{prn},'
          'execution time:{duration}s'.format(clf_name=clf_name,roc=roc,prn=prn,duration=duration))
    
        roc_list.append(roc)
        prn_list.append(prn)
    
    temp_df=pd.DataFrame(time_list).transpose()
    temp_df.columns=df_columns
    time_df=pd.concat([time_df,temp_df],axis=0)

    temp_df=pd.DataFrame(roc_list).transpose()
    temp_df.columns=df_columns
    roc_df=pd.concat([roc_df,temp_df],axis=0)

    temp_df=pd.DataFrame(prn_list).transpose()
    temp_df.columns=df_columns
    prn_df=pd.concat([prn_df,temp_df],axis=0)


.....Processing arrhythmia.mat ....
Angle-based Outlier Detector(ABOD) ROC:0.7687,precision @ rank n:0.3571,execution time:0.6188s
Cluster-based Local Outlier Factor ROC:0.7684,precision @ rank n:0.4643,execution time:0.6053s
Feature Bagging ROC:0.7799,precision @ rank n:0.5,execution time:1.464s
Histogram-base Outlier Detection(HBOS) ROC:0.8511,precision @ rank n:0.5714,execution time:0.2806s
Isolation Forest ROC:0.8527,precision @ rank n:0.5714,execution time:1.7665s
K Nearest Neighbors(KNN) ROC:0.782,precision @ rank n:0.5,execution time:0.3184s
Local Outlier Factor(LOF) ROC:0.7787,precision @ rank n:0.4643,execution time:0.1875s


C:\Users\Deepika\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant(MCD) ROC:0.8228,precision @ rank n:0.4286,execution time:2.9137s
One-class SVM(OCSVM) ROC:0.7986,precision @ rank n:0.5,execution time:0.0781s
Principal Component Analysis(PCA) ROC:0.7997,precision @ rank n:0.5,execution time:0.1882s

.....Processing cardio.mat ....
Angle-based Outlier Detector(ABOD) ROC:0.5763,precision @ rank n:0.1875,execution time:1.784s
Cluster-based Local Outlier Factor ROC:0.8221,precision @ rank n:0.4844,execution time:0.6308s
Feature Bagging ROC:0.4879,precision @ rank n:0.1406,execution time:2.2128s
Histogram-base Outlier Detection(HBOS) ROC:0.8453,precision @ rank n:0.4688,execution time:0.0312s
Isolation Forest ROC:0.9414,precision @ rank n:0.5,execution time:1.4969s
K Nearest Neighbors(KNN) ROC:0.6959,precision @ rank n:0.2812,execution time:0.5504s
Local Outlier Factor(LOF) ROC:0.4715,precision @ rank n:0.125,execution time:0.2694s


C:\Users\Deepika\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant(MCD) ROC:0.8778,precision @ rank n:0.3906,execution time:2.021s
One-class SVM(OCSVM) ROC:0.9507,precision @ rank n:0.5938,execution time:0.1719s
Principal Component Analysis(PCA) ROC:0.9638,precision @ rank n:0.6875,execution time:0.0446s

.....Processing glass.mat ....
Angle-based Outlier Detector(ABOD) ROC:0.7104,precision @ rank n:0.25,execution time:0.1875s
Cluster-based Local Outlier Factor ROC:0.8506,precision @ rank n:0.25,execution time:0.1618s
Feature Bagging ROC:0.7043,precision @ rank n:0.25,execution time:0.1307s
Histogram-base Outlier Detection(HBOS) ROC:0.6524,precision @ rank n:0.0,execution time:0.0155s
Isolation Forest ROC:0.7195,precision @ rank n:0.25,execution time:1.0123s
K Nearest Neighbors(KNN) ROC:0.7805,precision @ rank n:0.25,execution time:0.0313s
Local Outlier Factor(LOF) ROC:0.7774,precision @ rank n:0.25,execution time:0.0s
Minimum Covariance Determinant(MCD) ROC:0.7165,precision @ rank n:0.0,execution time:0.2359s
One-class 

...

C:\Users\Deepika\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:644: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant(MCD) ROC:0.3486,precision @ rank n:0.0,execution time:4.7878s
One-class SVM(OCSVM) ROC:0.4972,precision @ rank n:0.0,execution time:3.0796s
Principal Component Analysis(PCA) ROC:0.504,precision @ rank n:0.0,execution time:0.1403s

.....Processing pendigits.mat ....
Angle-based Outlier Detector(ABOD) ROC:0.7008,precision @ rank n:0.0308,execution time:6.653s
Cluster-based Local Outlier Factor ROC:0.9609,precision @ rank n:0.3077,execution time:1.0578s
Feature Bagging ROC:0.4687,precision @ rank n:0.0462,execution time:11.1659s
Histogram-base Outlier Detection(HBOS) ROC:0.9294,precision @ rank n:0.2615,execution time:0.0313s
Isolation Forest ROC:0.9422,precision @ rank n:0.2769,execution time:2.6523s
K Nearest Neighbors(KNN) ROC:0.7602,precision @ rank n:0.0462,execution time:2.6603s
Local Outlier Factor(LOF) ROC:0.481,precision @ rank n:0.0462,execution time:1.3563s
Minimum Covariance Determinant(MCD) ROC:0.8271,precision @ rank n:0.0615,execution time:7.9

C:\Users\Deepika\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.337315560576357 > -78.193487063766270). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\Deepika\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.088727288180465 > -77.017915011470095). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\Deepika\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.582785491594180 > -76.896496195181470). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning

C:\Users\Deepika\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.312868452035943 > -77.854429057839809). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\Deepika\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-50.387190804501891 > -77.157256658554772). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)
C:\Users\Deepika\anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:170: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-51.145180878067755 > -77.958300380489987). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning

Minimum Covariance Determinant(MCD) ROC:0.9903,precision @ rank n:0.7534,execution time:44.3937s
One-class SVM(OCSVM) ROC:0.9922,precision @ rank n:0.9553,execution time:156.1119s
Principal Component Analysis(PCA) ROC:0.9902,precision @ rank n:0.9503,execution time:0.1249s

.....Processing vertebral.mat ....
Angle-based Outlier Detector(ABOD) ROC:0.2797,precision @ rank n:0.0,execution time:0.2304s
Cluster-based Local Outlier Factor ROC:0.3908,precision @ rank n:0.0,execution time:0.1717s
Feature Bagging ROC:0.3027,precision @ rank n:0.0,execution time:0.1271s
Histogram-base Outlier Detection(HBOS) ROC:0.2695,precision @ rank n:0.0,execution time:0.0154s
Isolation Forest ROC:0.3576,precision @ rank n:0.0,execution time:1.0289s
K Nearest Neighbors(KNN) ROC:0.318,precision @ rank n:0.0,execution time:0.0312s
Local Outlier Factor(LOF) ROC:0.318,precision @ rank n:0.0,execution time:0.0s
Minimum Covariance Determinant(MCD) ROC:0.3321,precision @ rank n:0.0,execution time:0.183s
One-class S

In [31]:
roc_df

,Data,#Samples,#Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,iForest,KNN,LOF,MCD,OCSVM,PCA
0,wbc,378,30,5.5556,0.9181,0.9259,0.9555,0.9594,0.9355,0.9426,0.9536,0.9233,0.9433,0.9329
0,wbc,378,30,5.5556,0.9181,0.9259,0.9555,0.9594,0.9355,0.9426,0.9536,0.9233,0.9433,0.9329
0,arrhythmia,378,30,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,378,30,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,378,30,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,378,30,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,378,30,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,378,30,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,mnist,378,30,9.2069,0.7813,0.8447,0.7259,0.5675,0.7801,0.8409,0.7085,0.863,0.8417,0.8396
0,musk,378,30,3.1679,0.0809,1,0.5228,0.9999,0.9996,0.7348,0.5323,1,1,1


In [32]:
prn_df

,Data,#Samples,#Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,iForest,KNN,LOF,MCD,OCSVM,PCA
0,wbc,378,30,5.5556,0.5455,0.5455,0.7273,0.6364,0.6364,0.7273,0.6364,0.5455,0.6364,0.6364
0,wbc,378,30,5.5556,0.5455,0.5455,0.7273,0.6364,0.6364,0.7273,0.6364,0.5455,0.6364,0.6364
0,arrhythmia,378,30,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,cardio,378,30,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,378,30,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,378,30,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,378,30,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,378,30,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,mnist,378,30,9.2069,0.3562,0.4007,0.3664,0.1199,0.2979,0.4144,0.339,0.3973,0.3801,0.3767
0,musk,378,30,3.1679,0.0333,1,0.1667,0.9667,0.9333,0.2333,0.1333,0.9667,1,1


In [33]:
time_df

,Data,#Samples,#Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,iForest,KNN,LOF,MCD,OCSVM,PCA
0,wbc,378,30,5.5556,6.9853,6.0926,0.2602,6.0999,1.3578,0.0602,0.0156,1.215,0.0406,0.2428
0,wbc,378,30,5.5556,0.3531,0.2515,0.2317,0.0369,1.199,0.0627,0.0313,0.2684,0.0156,0
0,arrhythmia,378,30,14.6018,0.6188,0.6053,1.464,0.2806,1.7665,0.3184,0.1875,2.9137,0.0781,0.1882
0,cardio,378,30,9.6122,1.784,0.6308,2.2128,0.0312,1.4969,0.5504,0.2694,2.021,0.1719,0.0446
0,glass,378,30,4.2056,0.1875,0.1618,0.1307,0.0155,1.0123,0.0313,0,0.2359,0,0.0312
0,ionosphere,378,30,35.8974,0.3218,0.1873,0.2413,0.0409,1.1317,0.0624,0.0313,0.3571,0.0156,0.0469
0,letter,378,30,6.25,1.6129,0.4756,2.0918,0.0312,1.5386,0.5367,0.2687,4.3446,0.1563,0.0313
0,lympho,378,30,4.0541,0.1334,0.1544,0.1094,0.0156,1.0048,0.0156,0,0.1094,0,0.0128
0,mnist,378,30,9.2069,24.3983,3.3565,157.334,0.2309,5.9039,21.0592,18.9738,12.0949,12.0401,0.4356
0,musk,378,30,3.1679,9.5043,1.5278,50.2992,0.2398,4.1894,7.0172,6.5533,46.3545,2.6062,0.7582
